In [20]:
# | default_exp classes.DomoGroup

In [21]:
# | export
from domolibrary.routes.group import (
    SearchGroups_Error,
    Group_CRUD_Error,
    GroupType_Enum,
)

In [22]:
# | exporti
from dataclasses import dataclass, field
from typing import List

import httpx

from nbdev.showdoc import patch_to


import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.group as group_routes

import domolibrary.classes.DomoGroup_Membership as dmgm
import domolibrary.classes.DomoUser as dmu

In [23]:
# | hide
from nbdev.showdoc import show_doc
import os
import pandas as pd
import datetime as dt

# Domo Group


In [24]:
# | export
@dataclass
class DomoGroup:
    auth: dmda.DomoAuth = field(repr=False, default=None)
    id: str = None
    name: str = None
    type: str = None
    description: str = None
    members_id_ls: list[str] = field(repr=False, default_factory=list)
    owner_id_ls: list[str] = field(repr=False, default_factory=list)

    members_ls: list[dict] = field(repr=False, default_factory=list)
    owner_ls: list[dict] = field(repr=False, default_factory=list)

    custom_attributes: dict = field(default_factory=dict)

    def __post_init__(self):
        # self.domo_instance = self.domo_instance or auth.domo_instance
        self.Membership = dmgm.GroupMembership(self)

    @classmethod
    def _from_group_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.id or dd.groupId,
            name=dd.name,
            description=dd.description,
            type=dd.type or dd.groupType,
            members_id_ls=dd.userIds,
            owner_ls=dd.owners,
        )

    @classmethod
    def _from_grouplist_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.groupId,
            name=dd.name,
            description=dd.description,
            type=dd.groupType,
            owner_ls=dd.owners,
            owner_id_ls=[owner.id for owner in dd.owners],
            members_ls=dd.groupMembers,
            members_id_ls=[member.id for member in dd.members],
        )

    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth) -> List[dict]:
        domo_groups = [
            DomoGroup._from_group_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

        return domo_groups

## GET Methods


In [25]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    res = await group_routes.get_group_by_id(
        auth=auth, group_id=group_id, debug_api=debug_api, session=session
    )
    if return_raw:
        return res

    if res.status != 200:
        raise Exception()

    dg = cls._from_group_json(auth=auth, json_obj=res.response)

    # await dg.Membership.get_owners()
    # await dg.Membership.get_members() # disabled because causes recursion
    return dg

#### sample get_by_id


In [26]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

res = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id, return_raw=False)

res

warning this token has not been validated by who_am_i, run get_auth_token first


DomoGroup(id=2146122228, name='test_DO_NOT_DELETE', type='closed', description='updated via upsert_group on 2024-03-23', custom_attributes={})

In [27]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def search_by_name(
    cls,
    auth: dmda.DomoAuth,
    group_name: str,
    is_exact_match: bool = True,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    res = await group_routes.search_groups_by_name(
        auth=auth,
        search_name=group_name,
        debug_api=debug_api,
        is_exact_match=is_exact_match,
        session=session,
    )

    if return_raw:
        return res

    if isinstance(res.response, list):
        return cls._groups_to_domo_group(res.response, auth)

    return cls._from_group_json(auth=auth, json_obj=res.response)

#### sample search_by_name


In [28]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2q32"

try:
    res = await DomoGroup.search_by_name(group_name=group_name, auth=token_auth)
    print(res)

except SearchGroups_Error as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  SearchGroups_Error 🛑 - function: search_groups_by_name || status 200 || There is no exact match for Test Group 2q32 at domo-community


## GET Methods

#### sample get_owners


In [29]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group = DomoGroup(id=1781661643, auth=token_auth)

await group.Membership.get_owners()

print(group.owner_id_ls)  # does not differentiate between users and groups

group.owner_ls

warning this token has not been validated by who_am_i, run get_auth_token first
['1781661643', '822382906', '1893952720']


[DomoGroup(id=1781661643, name='TestMembership', type='open', description='group with membership', custom_attributes={}),
 DomoGroup(id=822382906, name='Grant: Manage all groups', type='system', description='Create, edit and delete any groups. Add and remove people from any group.', custom_attributes={}),
 DomoUser(id='1893952720', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=None)]

#### sample get_members


In [30]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group = DomoGroup(id=1781661643, auth=token_auth)

await group.Membership.get_members()

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoUser(id='696468809', display_name='Abish Srinath', email_address='abish.srinath@domo.com', role_id=2097317660, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=None),
 DomoUser(id='1345737456', display_name='Aditya Jain', email_address='jain_aditya95@yahoo.com', role_id=2, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=None),
 DomoUser(id='1141078945', display_name='Justin Pumford', email_address='jpumford@gmail.com', role_id=2097317660, department=None, title=None, avatar_key=None, phone_number=None, web_landin

## CRUD Methods

#### sample add_members


In [31]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old membership {', '.join([str(id) for id in domo_group.members_id_ls])}"
)


new_membership = await dmu.DomoUsers.by_id(
    user_ids=[1141078945, 1833256765], auth=token_auth, only_allow_one=False
)
await domo_group.Membership.add_members(add_user_ls=new_membership, debug_prn=False)

for user in new_membership:
    if user in domo_group.members_ls:
        print(f"user {user.id} - {user.display_name} successfully added")

warning this token has not been validated by who_am_i, run get_auth_token first
validate old membership 1141078945, 1725780094
user 1833256765 - Aaron Schofield successfully added
user 1141078945 - Justin Pumford successfully added


#### sample remove_members


In [32]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old membership {', '.join([str(id) for id in domo_group.members_id_ls])}"
)


remove_membership = await dmu.DomoUsers.by_id(
    user_ids=[1542225148], auth=token_auth, only_allow_one=False
)
await domo_group.Membership.remove_members(
    remove_user_ls=remove_membership, debug_prn=False
)

for user in remove_membership:
    if user not in domo_group.members_ls:
        print(f"user {user.id} - {user.display_name} successfully removed")

warning this token has not been validated by who_am_i, run get_auth_token first
validate old membership 1141078945, 1725780094, 1833256765
user 1542225148 - Adam Landefeld successfully removed


#### sample set_members


In [33]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old membership {', '.join([str(id) for id in domo_group.members_id_ls])}"
)


set_membership = await dmu.DomoUsers.by_id(
    user_ids=[
        1141078945,
        1542225148,
        1725780094,
    ],
    auth=token_auth,
    only_allow_one=False,
)
await domo_group.Membership.set_members(user_ls=set_membership, debug_prn=False)

test_members = set_membership
for member in domo_group.members_ls:
    if member not in test_members:
        test_members.append(member)

for user in test_members:
    if user not in set_membership:
        print(f"user {user.id} - {user.display_name} does not belong")

    if user in set_membership:
        print(f"user {user.id} - {user.display_name} belongs")

warning this token has not been validated by who_am_i, run get_auth_token first
validate old membership 1141078945, 1725780094, 1833256765
user 1542225148 - Adam Landefeld belongs
user 1725780094 - Brett Gibson belongs
user 1141078945 - Justin Pumford belongs


In [34]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

membership = [
    dmu.DomoUser(id=1141078945, auth=token_auth),
    dmu.DomoUser(id=1725780094, auth=token_auth),
]


domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(f"validate membership {', '.join([str(id) for id in domo_group.members_id_ls])}")

# await domo_group.Membership.get_members()

await domo_group.Membership.set_members(user_ls=membership, debug_prn=False)

warning this token has not been validated by who_am_i, run get_auth_token first
validate membership 1141078945, 1542225148, 1725780094


[DomoUser(id='1725780094', display_name='Brett Gibson', email_address='Brett.Gibson@domo.com', role_id=2, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=None),
 DomoUser(id='1141078945', display_name='Justin Pumford', email_address='jpumford@gmail.com', role_id=2097317660, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=None)]

#### sample add_owners


In [35]:
# | eval :false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)
await domo_group.Membership.get_owners()

print(f"validate old ownership {', '.join([str(id) for id in domo_group.owner_id_ls])}")


new_ownership_user = await dmu.DomoUsers.by_id(
    user_ids=[1345408759], auth=token_auth, only_allow_one=False
)
new_ownership_group = await DomoGroup.get_by_id(group_id=1028365465, auth=token_auth)
new_ownership = new_ownership_user + [new_ownership_group]

try:
    await domo_group.Membership.add_owners(add_owner_ls=new_ownership, debug_prn=False)

    for user in new_ownership:
        if user in domo_group.owner_ls:
            if isinstance(user, dmu.DomoUser):
                print(f"user {user.id} - {user.display_name} successfully added")
            if isinstance(user, DomoGroup):
                print(f"user {user.id} - {user.name} successfully added")
                
except Exception as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
validate old ownership 1028365465, 822382906
🛑  UpdateGroupMembership 🛑 - function: unable to add Team Alpha to test_DO_NOT_DELETE at domo-community


#### sample remove_owners


In [36]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(f"validate old ownership {', '.join([str(id) for id in domo_group.owner_id_ls])}")

remove_owner = await dmu.DomoUsers.by_id(
    user_ids=[797235532], auth=token_auth, only_allow_one=False
)

await domo_group.Membership.remove_owners(remove_owner_ls=remove_owner, debug_prn=False)


for user in remove_owner:
    if user not in domo_group.owner_ls:
        print(f"user {user.id} - {user.display_name} successfully removed")

warning this token has not been validated by who_am_i, run get_auth_token first
validate old ownership 
user 797235532 - Teddy Murphy successfully removed


#### set_ownership


In [37]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old ownership {', '.join([str(id) for id in domo_group.members_id_ls])}"
)


set_ownership_user = await dmu.DomoUsers.by_id(
    user_ids=[797235532], auth=token_auth, only_allow_one=False
)
set_ownership_group = await ce.gather_with_concurrency(
    n=60,
    *[
        DomoGroup.get_by_id(group_id=group_id, auth=token_auth)
        for group_id in [1028365465, 822382906]
    ],
)
set_ownership = set_ownership_user + set_ownership_group

try:
    await domo_group.Membership.set_owners(owner_ls=set_ownership, debug_prn=False)

    test_owners = set_ownership
    for owner in domo_group.owner_ls:
        if owner not in test_owners:
            test_owners.append(owner)

    for owner in test_owners:
        if owner not in set_ownership:
            print(
                f"{type(owner).__name__} {owner.id} - {getattr(owner, 'display_name', None) or getattr(owner, 'name', None)} does not belong"
            )

        if user in set_ownership:
            print(
                f"{type(owner).__name__} {owner.id} - {getattr(owner, 'display_name', None) or getattr(owner, 'name', None)} belongs"
            )

except Exception as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
validate old ownership 1141078945, 1725780094
🛑  UpdateGroupMembership 🛑 - function: unable to add Team Alpha to test_DO_NOT_DELETE at domo-community


# Domo Groups


In [38]:
# | export
class DomoGroups:
    def __init__(self):
        pass

    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth):
        return [
            DomoGroup._from_group_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

### get_all_groups


In [39]:
# | export
@patch_to(DomoGroups, cls_method=True)
async def get_all_groups(
    cls: DomoGroups,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    res = await group_routes.get_all_groups(
        auth=auth, debug_api=debug_api, session=session
    )

    if len(res.response) > 0:
        json_list = res.response

        return cls._groups_to_domo_group(json_list=json_list, auth=auth)

    else:
        return []

#### sample implementation of get_all_groups


In [40]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_groups = await DomoGroups.get_all_groups(auth=token_auth)

warning this token has not been validated by who_am_i, run get_auth_token first


### Toggle Group Visibility


In [41]:
# |exporti


@patch_to(DomoGroups, cls_method=True)
async def toggle_system_group_visibility(
    cls: DomoGroups,
    auth: dmda.DomoAuth,
    is_hide_system_groups: bool,
    debug_api: bool = False,
):
    return await group_routes.toggle_system_group_visibility(
        auth=auth, is_hide_system_groups=is_hide_system_groups, debug_api=debug_api
    )

#### sample toggle_system_group_visibility


In [42]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoGroups.toggle_system_group_visibility(
    auth=token_auth, is_hide_system_groups=False
)

all_groups_with_hidden = await DomoGroups.get_all_groups(auth=token_auth)

await DomoGroups.toggle_system_group_visibility(
    auth=token_auth, is_hide_system_groups=True
)

all_groups = await DomoGroups.get_all_groups(auth=token_auth)


print(
    f"there are {len(all_groups)} standard groups, and {len(all_groups_with_hidden)} groups including system groups"
)

toggling group visiblity in domo-community show system groups
warning this token has not been validated by who_am_i, run get_auth_token first
toggling group visiblity in domo-community hiding system groups
there are 120 standard groups, and 47 groups including system groups


# CRUD Group


In [43]:
# | eval: false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group ac"

domo_group = await DomoGroup.search_by_name(auth=token_auth, group_name=group_name)
await domo_group.Membership.add_owner_manage_groups_role()

warning this token has not been validated by who_am_i, run get_auth_token first
toggling group visiblity in domo-community show system groups
toggling group visiblity in domo-community hiding system groups


In [44]:
# | exporti


@patch_to(DomoGroup, cls_method=True)
async def create_from_name(
    cls: DomoGroup,
    auth: dmda.DomoAuth,
    group_name: str = None,
    group_type: str = "open",  # use GroupType_Enum
    description: str = None,
    is_include_manage_groups_role: bool = True,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    res = await group_routes.create_group(
        auth=auth,
        group_name=group_name,
        group_type=group_type,
        description=description,
        debug_api=debug_api,
    )

    domo_group = cls._from_group_json(auth=auth, json_obj=res.response)

    await domo_group.Membership.add_owner_manage_groups_role()

    return domo_group

#### sample create_from_name


In [45]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group ac"

try:
    res = await DomoGroup.create_from_name(group_name=group_name, auth=token_auth)
    print(res)
except group_routes.Group_CRUD_Error as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  Group_CRUD_Error 🛑 - function: create_group || status 400 || Test Group ac already exists. Choose a different group_name at domo-community


## update metadata


In [46]:
# | exporti


@patch_to(DomoGroup)
async def update_metadata(
    self: DomoGroup,
    auth: dmda.DomoAuth = None,
    group_name: str = None,
    group_type: str = None,  # use GroupType_Enum
    description: str = None,
    additional_params: dict = None,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    try:
        res = await group_routes.update_group(
            auth=auth,
            group_id=self.id,
            group_name=group_name,
            group_type=group_type,
            description=description,
            additional_params=additional_params,
            debug_api=debug_api,
            session=session,
        )

        if return_raw:
            return res

        updated_group = await DomoGroup.get_by_id(auth=auth, group_id=self.id)

        self.name = updated_group.name or self.name
        self.description = updated_group.description or self.description
        self.type = updated_group.type or self.type

    except Group_CRUD_Error as e:
        if group_type != self.type:
            raise Group_CRUD_Error(
                status=400,
                message=f"probably cannot change group_type to '{group_type}' from current type '{self.type}' consider passing `addtional_parameters`",
                domo_instance=auth.domo_instance,
                function_name="update_group",
                parent_class=self.__class__.__name__,
            )

        else:
            raise e

    return self

#### sample update_metadata


In [47]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

description = f"update metadata - updated {dt.date.today()}"

domo_group = await DomoGroup.get_by_id(group_id=group_id, auth=token_auth)

await domo_group.update_metadata(description=description, return_raw=False)

warning this token has not been validated by who_am_i, run get_auth_token first


DomoGroup(id=2146122228, name='test_DO_NOT_DELETE', type='closed', description='update metadata - updated 2024-03-23', custom_attributes={})

In [48]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = "1513712315"

domo_group = await DomoGroup.get_by_id(group_id=group_id, auth=token_auth)

try:
    await domo_group.update_metadata(
        auth=token_auth,
        # group_name="Test Group ABCs",
        group_type="open",
        description=f"updated via API on {dt.date.today()}",
        debug_api=False,
    )

except Group_CRUD_Error as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  Group_CRUD_Error 🛑 - function: DomoGroup.update_group || status 400 || probably cannot change group_type to 'open' from current type 'closed' consider passing `addtional_parameters` at domo-community


## Upsert Groups


In [49]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def upsert(
    cls: DomoGroup,
    auth: dmda.DomoAuth,
    group_name: str,
    group_type: str = None,  # if create_group, use routes.class.GroupType_Enum
    description: str = None,
    debug_api: bool = False,
    debug_prn: bool = False,
    additional_params: dict = None,
    session: httpx.AsyncClient = None,
):
    domo_group = None

    try:
        domo_group = await DomoGroup.search_by_name(
            auth=auth, group_name=group_name, is_exact_match=True, session=session
        )

        return await domo_group.update_metadata(
            group_type=group_type, description=description, debug_api=debug_api
        )

    except group_routes.SearchGroups_Error as e:
        print(e)
        return await DomoGroup.create_from_name(
            auth=auth,
            group_name=group_name,
            group_type=group_type,
            description=description,
            debug_api=debug_api,
            session=session,
        )

    except Group_CRUD_Error as e:
        print(e)

        # try running without changing group type

        return await group_routes.update_group(
            auth=auth,
            group_id=domo_group.id,
            group_name=group_name,
            # group_type=group_type,
            description=description,
            additional_params=additional_params,
            debug_api=debug_api,
            session=session,
        )

#### sample upsert


In [50]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "test_DO_NOT_DELETE"
description = f"updated via upsert_group on {dt.date.today()}"

await DomoGroup.upsert(
    group_name=group_name, auth=token_auth, description=description, debug_api=False
)

warning this token has not been validated by who_am_i, run get_auth_token first


DomoGroup(id=2146122228, name='test_DO_NOT_DELETE', type='closed', description='updated via upsert_group on 2024-03-23', custom_attributes={})

## delete account


In [51]:
# | exporti
@patch_to(DomoGroup)
async def delete(
    self: DomoGroup,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: httpx.AsyncClient = None,
):
    res = await group_routes.delete_groups(
        auth=self.auth,
        group_ids=[str(self.id)],
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
        session=session,
    )

    res.parent_class = self.__class__.__name__

    return res

In [52]:
show_doc(DomoGroup.delete)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoGroup.py#L325){target="_blank" style="float:right; font-size:smaller"}

### DomoGroup.delete

>      DomoGroup.delete (debug_api:bool=False, debug_num_stacks_to_drop=2,
>                        session:httpx.AsyncClient=None)

In [53]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    domo_group = await DomoGroup.upsert(
        auth=token_auth,
        group_name="hello world",
        description=f"updated via API on {dt.date.today()}",
    )

    await domo_group.delete(debug_api=False)

except Exception as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  SearchGroups_Error 🛑 - function: search_groups_by_name || status 200 || There is no exact match for hello world at domo-community
toggling group visiblity in domo-community show system groups


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()